<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/Audio/Fine_Tuning_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/Audio

/content/drive/MyDrive/Colab Notebooks/NLP/Audio


In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install accelerate -q

In [ ]:
!pip install --upgrade datasets transformers soundfile librosa evaluate jiwer tensorboard gradio

In [5]:
from huggingface_hub import notebook_login

notebook_login()

-----

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sw", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sw", split="test", use_auth_token=True)

In [7]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 36847
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10238
    })
})


In [8]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [9]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [10]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swahili", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [12]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Uko katika pembe la kusini-mashariki kabisa la nchi.
Decoded w/ special:    <|startoftranscript|><|sw|><|transcribe|><|notimestamps|>Uko katika pembe la kusini-mashariki kabisa la nchi.<|endoftext|>
Decoded w/out special: Uko katika pembe la kusini-mashariki kabisa la nchi.
Are equal:             True


In [13]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swahili", task="transcribe")

In [14]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/d0c515954317076cb4654c80caae844d8922490cb28ecb53f2df4b52ab7baa52/sw_train_0/common_voice_sw_28660554.mp3', 'array': array([ 5.52202634e-29, -2.20881053e-29, -4.57539325e-29, ...,
        1.07723235e-06, -3.07742812e-06, -4.13509224e-06]), 'sampling_rate': 48000}, 'sentence': 'Uko katika pembe la kusini-mashariki kabisa la nchi.'}


In [15]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/d0c515954317076cb4654c80caae844d8922490cb28ecb53f2df4b52ab7baa52/sw_train_0/common_voice_sw_28660554.mp3', 'array': array([-8.52651283e-14, -1.35003120e-13, -9.94759830e-14, ...,
        4.05998435e-06, -4.27663326e-06, -2.08104029e-06]), 'sampling_rate': 16000}, 'sentence': 'Uko katika pembe la kusini-mashariki kabisa la nchi.'}


In [17]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/36847 [00:00<?, ? examples/s]